In [111]:
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
import sunpy.map
from astropy import units as u
import sunpy.coordinates.transformations
from sunpy.coordinates import frames
from pathlib import Path

import requests


In [28]:
class harp2noaa():
    """
    A class used to represent an harp2noaa object

    ...

    Attributes
    ----------
    fsave : str
        Filename for base convertor txt

    Methods
    -------
    update_dataset()
        Fetch newest list from jsoc
        
    harp2noaa(NOAANUM)
        Returns HAPRNUM for given NOAA.
        Could be int or str
    """
    def __init__(self, fsave='./HARP_TO_NOAA.txt'):
        """
        Parameters
        ----------
        name : str
            The name of the animal
        num_legs : int, optional
            The number of legs the animal (default is 4)
        """
        self.fsave = fsave
        if not os.path.exists(self.fsave):
            print("Default HARP_TO_NOAA DOES NOT exists, fetching")
            self.update_dataset()
        else:
            print("Default HARP_TO_NOAA exists")
        print("Loading file")
        self._load_dataset()        
    
    def _load_dataset(self):
        """
        Load dataset into content variable
        """
        with open(self.fsave) as f:
            content = [line.rstrip() for line in f]
        self.content = content
    
    def update_dataset(self):
        """
        Wrapper method for download newest harpnum to noaa list
        """
        url = 'http://jsoc.stanford.edu/doc/data/hmi/harpnum_to_noaa/all_harps_with_noaa_ars.txt'
        r = requests.get(url)
        if r.ok:
            print("Content fetched, saving...")
            with open(self.fsave, 'wb') as f:
                f.write(r.content)
                print("File saved to {}".format(self.fsave))
            print("Loading new dataset")
            self._load_dataset()
            
    def harp2noaa(self, NOAANUM):
        '''
        This returns HARP number for provided NOAA region from content file
        '''
        NOAANUM = str(NOAANUM)
        ins = [index for index, string in enumerate(self.content) if NOAANUM in string]
        harpnum = [ self.content[x].split(' ')[0] for x in ins ]
        if len(harpnum) != 1:
            raise Exception("Be careful! Your region is over multiple HARPs or it was not found! HARPS: {}".format(harpnum))
        return int(harpnum[0])

In [29]:
a = harp2noaa()
a.update_dataset()
HARPNUM = a.harp2noaa("11950")
print(HARPNUM)

Default HARP_TO_NOAA exists
Loading file
Content fetched, saving...
File saved to ./HARP_TO_NOAA.txt
Loading new dataset
3604


In [137]:
#Ok, i dont know how to work with class factories so i cant extend this
class process_continuum():
    """
    A class used to represent an process_continuum object.
    We use this object to automate continuum images processing
    
    #### WE ARE NOT LOADING ALL FILES INTO THIS OBJECT, WE ARE GOING ONE BY ONE!!!! ####

    ...

    Attributes
    ----------
    inpath : str
        Path to fits file that we need to process
    
    opath : str
        Path where corrected fits should be saved

    Methods
    -------
    __limb_dark()
        limbdarkening coeeficients
    
    _correct_for_limb()
        Function used to correct for limb darkening
        
    _normalize():
        Function used to normalize data
        
        
        
    master_wrap
    """
    
    def __init__(self, inpath, overwrite=False):
        self.inpath = inpath
        self.overwrite = overwrite
        # check if input is list
        if isinstance(self.inpath, list):
            print("Dataset detected!")
            #self.outpath = os.path.dirname(self.inpath.replace('raw','processed'))
            #print("Files will be saved to: {}".format(self.outpath_dir)) 
            #Path(os.path.dirname(self.inpath.replace('raw','processed'))).mkdir(parents=True, exist_ok=True)
            print("Processing ...")
            for in_img in self.inpath:
                Path(os.path.dirname(in_img.replace('raw','processed'))).mkdir(parents=True, exist_ok=True)
                out = self._master_wrap(in_img)
                out.save(in_img.replace('raw','processed'))
            print("Processing done.")
        elif isinstance(self.inpath, str):
            print("Single file detected!")
            print("Processing ...")
            out = self._master_wrap(self.inpath)
            print("Processing done.")
            self.outpath = self.inpath.replace('raw','processed')
            print("File will be saved to: {}".format(self.outpath)) 
            Path(os.path.dirname(self.outpath)).mkdir(parents=True, exist_ok=True)
            out.save(self.outpath)
            print("File saved!")
        else:
            raise ValueError("I cant understand input, should be string or list of strings")
            
        
    
    def __limb_dark(self, r, koef=np.array([0.32519, 1.26432, -1.44591, 1.55723, -0.87415, 0.173333])):
        """
        This function takes r as distance from sun center in units of sun radii and return division factor for correction.
        We are making it internal function, because we only need to call it from correct_for_limb function.
        """
        # r is normalized distance from center [0,1]
        if len(koef) != 6:
            raise ValueErrror("koef len should be exactly 6")
        if np.max(r) > 1 or np.min(r) < 0:
            raise ValueError("r should be in [0,1] range")
        mu = np.sqrt(1-r**2)  # mu = cos(theta)
        return koef[0]+koef[1]*mu+koef[2]*mu**2+koef[3]*mu**3+koef[4]*mu**4+koef[5]*mu**5

    def _correct_for_limb(self, sunpy_map):
        """
        This function takes sunpy map and removes limb darkening from it
        It transfer coordinate mesh to helioprojective coordinate (using data from header)
        Calucalates distance from sun center in units of sun radii at the time of observation
        Uses limb_dark function with given coeffitiens and divides by that value

        Input: sunpy_map (sunpy.map) - input data
        Returns: sunpy.map - output data object
        """
        helioproj_limb = sunpy.map.all_coordinates_from_map(sunpy_map).transform_to(
            frames.Helioprojective(observer=sunpy_map.observer_coordinate))
        rsun_hp_limb = sunpy_map.rsun_obs.value
        distance_from_limb = np.sqrt(
            helioproj_limb.Tx.value**2+helioproj_limb.Ty.value**2)/rsun_hp_limb
        limb_cor_data = sunpy_map.data / self.__limb_dark(distance_from_limb)
        return sunpy.map.Map(limb_cor_data, sunpy_map.meta)

    # AVERAGE
    def _normalize(self, sunpy_map, header_keyword='AVG_F_NO', NBINS=100):
        '''
        This function normalizes sunpy map
        It first creates histogram of data
        Finds maximum of histogram and divide whole dataset with that number
        This is efectevly normalization to quiet sun

        input:  sunpy_map (sunpy.map) - input data
                header_keyword (string) - name of header keyword in which maximum of histogram will be written to 
                                          This allows users to later on, revert to unnormalized image, default is AVG_F_NO
                NBINS (int) - How many bins you want for your histogram, default is 100
        output: sunpy.map - output data object
        '''
        weights, bin_edges = np.histogram(
            sunpy_map.data.flatten(), bins=NBINS, density=True)
        # MAGIC I SAY!
        # find maximum of histogram
        k = (weights == np.max(weights)).nonzero()[0][0]
        # find flux value for maximum of histogram
        I_avg = (bin_edges[k+1]+bin_edges[k])/2
        # update data
        I_new = sunpy_map.data/I_avg
        # create new keyword in header
        # AVG_F_ON
        # AVG_F_EN
        sunpy_map.meta[header_keyword] = I_avg
        # create new map
        return sunpy.map.Map(I_new, sunpy_map.meta)

    def _master_wrap(self,fname):
        '''
        This function is just simple wrapper for all provided functions

        input: filename (string) -  fits file path that correction shoud be performed on
        output: ofile (string) - string with path to new file
        '''
        # load data
        sunpy_data = sunpy.map.Map(fname)
        # correct map for limb
        mid_data = self._correct_for_limb(sunpy_data)
        # Normalize
        mid_data = self._normalize(mid_data, header_keyword='AVG_F_ON')
        return mid_data
        #mid_data.peek()
        # enhance
        #mid_data = enhance_wrapper(mid_data)
        # normalize again, enhance can make mess with flux
        #mid_data = normalize(mid_data, header_keyword='AVG_F_EN')
        # Create new filename
        
        #Create new logic for saving files
#         outfile = os.path.basename(filename).replace(
#             search_criterium, search_criterium+sufix)
#         ofile = os.path.join(output_dir, outfile)
        # save map
        #mid_data.save(ofile)
        #return ofile





#     def enhance_wrapper(sunpy_map, depth=5, model="keepsize", activation="relu", ntype="intensity"):
#         '''
#         This procedures run enhance https://github.com/cdiazbas/enhance (it works only from my fork https://github.com/lzivadinovic/enhance)
#         on input sunpy map
#         Check source code for explanation of code and input parameters

#         input: sunpy_map (sunpy.map) - input data set
#         output: sunpy.map - output data object (enhanced)
#         '''
#         # if rtype is spmap, there is no need for output, it will return sunpy.map object (lzivadinovic/enhance fork - master branch)
#         out = enhance(inputFile=sunpy_map, depth=depth, model=model,
#                       activation=activation, ntype=ntype, output='1.fits', rtype='spmap')
#         out.define_network()
#         return out.predict()

In [69]:
MY_MAIL='lazar.zivadinovic.994@gmail.com'
import os
# Lets make jsoc query 
from sunpy.net import jsoc
from sunpy.net import attrs as a
from sunpy.time import parse_time
#initialize client
client = jsoc.JSOCClient()

data_root='./data'
#DOWNLOAD_PATH_FOR_RAW_DATA
download_path=os.path.join(data_root,str(HARPNUM),'raw')

###### THIS IS THE REAL QUERY, LETS MAKE SOMETHING SMALLER FOR TESTING ########
# Create query
resjsoc = client.search(a.jsoc.PrimeKey('HARPNUM', HARPNUM),
                        a.jsoc.Series('hmi.sharp_cea_720s'),
                        a.jsoc.Segment('Bp') & a.jsoc.Segment('Bt') & 
                        a.jsoc.Segment('Br') & a.jsoc.Segment('continuum'))#,
                        #a.jsoc.Notify(MY_MAIL))

#Lets fetch few images for testing
#Note that client.jsoc does not support slice, so we need new query
#This is a way around it
T1 = resjsoc.table['T_REC'][0]
T2 = resjsoc.table['T_REC'][3]
resjsoc = client.search(a.jsoc.PrimeKey('HARPNUM', HARPNUM),
                        a.Time(T1,T2),
                        a.jsoc.Series('hmi.sharp_cea_720s'),
                        a.jsoc.Segment('Bp') & a.jsoc.Segment('Bt') & 
                        a.jsoc.Segment('Br') & a.jsoc.Segment('continuum'),
                        a.jsoc.Notify(MY_MAIL))

result = client.fetch(resjsoc, path=download_path, progress=True, wait=True)


Export request pending. [id="JSOC_20200425_263", status=2]
Waiting for 0 seconds...
16 URLs found for download. Full request totalling 7MB


In [70]:
print(result)

['data/3604/raw/hmi.sharp_cea_720s.3604.20140108_032400_TAI.continuum.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.Bp.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_032400_TAI.Bt.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_033600_TAI.continuum.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.Bt.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_034800_TAI.Bt.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_032400_TAI.Bp.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_034800_TAI.continuum.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.Br.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_033600_TAI.Bp.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_034800_TAI.Bp.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.continuum.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_033600_TAI.Br.fits', 'data/3604/raw/hmi.sharp_cea_720s.3604.20140108_032400_TAI.Br.fits', 'data

In [71]:
if result.errors:
    print("We had some errors, here is log")
    print(result.errors)
    print("Lets fetch them via requests library")
    for reserr in result.errors:
        r = requests.get(reserr.url)
        if r.ok:
            print("Content fetched, saving...")
        path_for_new_img = os.path.join(download_path,reserr.url.split('/')[-1])
        with open(path_for_new_img, 'wb') as f:
            f.write(r.content)
            print("File saved to {}".format(path_for_new_img))
    #we should have all results now, lets update results list
    result = glob.glob(os.path.join(download_path,'*'))

We had some errors, here is log
[error(filepath_partial=<function Downloader.enqueue_file.<locals>.filepath at 0x7ff9742340e0>, url='http://jsoc.stanford.edu/SUM66/D1283128204/S00000/hmi.sharp_cea_720s.3604.20140108_033600_TAI.Bt.fits', exception=ClientConnectorError(ConnectionKey(host='jsoc.stanford.edu', port=80, is_ssl=False, ssl=None, proxy=None, proxy_auth=None, proxy_headers_hash=None), ConnectionRefusedError(111, "Connect call failed ('171.64.103.244', 80)")))]
Lets fetch them via requests library
Content fetched, saving...
File saved to ./data/3604/raw/hmi.sharp_cea_720s.3604.20140108_033600_TAI.Bt.fits


In [72]:
# Ok, lets process continuum images
cont = sorted([ res for res in result if 'continuum' in res])
print(cont)

['./data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.continuum.fits', './data/3604/raw/hmi.sharp_cea_720s.3604.20140108_032400_TAI.continuum.fits', './data/3604/raw/hmi.sharp_cea_720s.3604.20140108_033600_TAI.continuum.fits', './data/3604/raw/hmi.sharp_cea_720s.3604.20140108_034800_TAI.continuum.fits']


In [42]:
asd = sunpy.map.Map(cont)

# def master_wrap(filename):
#     '''
#     This function is just simple wrapper for all privided functions
    
#     input: filename (string) -  fits file path that correction shoud be performed on
#     output: ofile (string) - string with path to new file
#     '''
#     # load data
#     sunpy_data = sunpy.map.Map(filename)
#     # correct map for limb
#     mid_data = correct_for_limb(sunpy_data)
#     # Normalize
#     mid_data = normalize(mid_data, header_keyword='AVG_F_ON')
#     # enhance
#     mid_data = enhance_wrapper(mid_data)
#     # normalize again, enhance can make mess with flux
#     mid_data = normalize(mid_data, header_keyword='AVG_F_EN')
#     # Create new filename
#     outfile = os.path.basename(filename).replace(
#         search_criterium, search_criterium+sufix)
#     ofile = os.path.join(output_dir, outfile)
#     # save map
#     mid_data.save(ofile)
#     return ofile





In [140]:
#os.path.join(*cont[0].split('/')[0:-2],'processed')
print(cont[0])
#process_continuum(cont)
a=sunpy.map.Map(cont[0])

./data/3604/raw/hmi.sharp_cea_720s.3604.20140108_031200_TAI.continuum.fits


In [141]:
dir(a)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_as_mpl_axes',
 '_base_name',
 '_coordinate_frame_name',
 '_data',
 '_default_carrington_longitude',
 '_default_dsun',
 '_default_heliographic_latitude',
 '_default_heliographic_longitude',
 '_default_time',
 '_fix_bitpix',
 '_fix_date',
 '_fix_naxis',
 '_get_cmap_name',
 '_get_lon_lat',
 '_mask',
 '_meta',
 '_new_instance',
 '_nickname',
 '_reference_latitude',
 '_reference_longitude',
 '_registry',
 '_remove_existing_observer_location',
 '_rotation_matrix_from_crota',
 '_shift',
 '_supported_observer_coordinates',
 '_uncertainty',
 '_unit',
 '_validate_meta',
 '_wcs